In [ ]:
import cv2
import os
import glob
import random
import shutil
from pathlib import Path


from ultralytics import YOLO
import torch

In [ ]:
# Load your custom model configuration
model = YOLO('yolo11n-bifpn.yaml')

# Print info to verify layers are there
model.info()

YOLO11n-bifpn summary: 200 layers, 2,599,933 parameters, 2,599,917 gradients, 6.6 GFLOPs


(200, 2599933, 2599917, 6.590592000000001)

In [ ]:
# --- 1. SMART PATH DETECTION ---
def find_source_root():
    """Automatically finds the 'segmentation_task' folder."""
    current_dir = Path.cwd()
    print(f"Searching for data starting from: {current_dir}")
    
    # Places to look
    candidates = [
        current_dir / "brisc2025" / "segmentation_task", # If running from outside
        current_dir / "segmentation_task",                # If running inside brisc2025
        current_dir                                       # If running inside segmentation_task
    ]
    
    for path in candidates:
        if (path / "train").exists():
            print(f"Found data at: {path}")
            return path
            
    # If not found, try recursive search (deep search)
    for path in current_dir.rglob("segmentation_task"):
        if path.is_dir() and (path / "train").exists():
            print(f"Found data at: {path}")
            return path
            
    raise FileNotFoundError("Could not find 'segmentation_task' folder with a 'train' subfolder.")

# --- 2. CONFIGURATION ---
try:
    SOURCE_ROOT = find_source_root()
except Exception as e:
    print(e)
    exit()

DEST_ROOT = Path("yolo_dataset")

# --- 3. CONVERSION LOGIC ---
def convert_mask_to_yolo_box(mask_path):
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    if mask is None: return None
    
    # Threshold & Contours
    _, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    labels = []
    h_img, w_img = mask.shape[:2]
    
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w < 3 or h < 3: continue # Filter tiny noise
            
        # Normalize (0.0 - 1.0)
        x_center = (x + w / 2) / w_img
        y_center = (y + h / 2) / h_img
        w_norm = w / w_img
        h_norm = h / h_img
        
        # Class 0 = Tumor
        labels.append(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}")
        
    return labels

def process_subset(subset_name):
    # Detect if folder is named 'image' or 'images'
    subset_path = SOURCE_ROOT / subset_name
    
    if (subset_path / "image").exists():
        img_dir_name = "image"
        mask_dir_name = "mask"
    elif (subset_path / "images").exists():
        img_dir_name = "images"
        mask_dir_name = "masks"
    else:
        print(f"Skipping {subset_name}. NOT FOUND!")
        return

    img_src = subset_path / img_dir_name
    mask_src = subset_path / mask_dir_name
    
    img_dst = DEST_ROOT / "images" / subset_name
    label_dst = DEST_ROOT / "labels" / subset_name
    
    img_dst.mkdir(parents=True, exist_ok=True)
    label_dst.mkdir(parents=True, exist_ok=True)
    
    # Get all images
    images = list(img_src.glob("*.jpg")) + list(img_src.glob("*.png")) + list(img_src.glob("*.jpeg"))
    
    print(f"Processing {subset_name}: found {len(images)} images in {img_src}")
    
    count = 0
    for img_path in images:
        # Copy Image
        shutil.copy(img_path, img_dst / img_path.name)
        
        # Find Mask (try jpg, png, and jpeg)
        possible_masks = [
            mask_src / img_path.name,
            mask_src / img_path.with_suffix('.png').name,
            mask_src / img_path.with_suffix('.jpg').name
        ]
        
        found_labels = False
        for mask_p in possible_masks:
            if mask_p.exists():
                yolo_labels = convert_mask_to_yolo_box(mask_p)
                if yolo_labels:
                    txt_name = img_path.with_suffix('.txt').name
                    with open(label_dst / txt_name, 'w') as f:
                        f.write('\n'.join(yolo_labels))
                    found_labels = True
                break
        
        # If no mask found, make empty label file (Background image)
        if not found_labels:
            txt_name = img_path.with_suffix('.txt').name
            with open(label_dst / txt_name, 'w') as f:
                pass 
        
        count += 1
        if count % 100 == 0:
            print(f"  Converted {count}...", end='\r')

if __name__ == "__main__":
    process_subset("train")
    process_subset("test")
    
    # Verify
    n_train = len(list((DEST_ROOT / "images/train").glob("*")))
    if n_train > 0:
        print(f"\n\n{n_train} images prepared in 'yolo_dataset'.")
    else:
        print("\nfolder is empty.")

Searching for data starting from: d:\Data Adji\Binus\Semester 5\DeepLearning\Project
✅ Found data at: d:\Data Adji\Binus\Semester 5\DeepLearning\Project\brisc2025\segmentation_task
Processing train: found 3933 images in d:\Data Adji\Binus\Semester 5\DeepLearning\Project\brisc2025\segmentation_task\train\images
Processing test: found 860 images in d:\Data Adji\Binus\Semester 5\DeepLearning\Project\brisc2025\segmentation_task\test\images
  Converted 800...

✅ SUCCESS! 3933 images prepared in 'yolo_dataset'.
You can now run 'python train_bifpn.py'


In [ ]:
def train_custom_model():
    print(f"Using Device: {torch.cuda.get_device_name(0)}")
    
    # 1. Initialize your Custom Architecture
    # This reads the YAML we fixed (with the correct channel sizes)
    model = YOLO('yolo11n-bifpn.yaml') 

    # 2. Transfer Learning
    # We load standard YOLOv11n weights. 
    # The backbone will match. The new BiFPN layers will start with random weights.
    try:
        print("Loading pre-trained weights...")
        model.load('yolo11n.pt') 
    except Exception as e:
        print("Note: Partial weight loading is expected because architecture changed.")

    # 3. Start Training
    # VRAM NOTE: We use batch=4 because 4GB VRAM is tight for BiFPN+CBAM.
    model.train(
        data='data.yaml',         # Points to your new yolo_dataset
        epochs=50,                # 50 epochs is a good start
        imgsz=640,                # Standard image size
        batch=4,                  # <--- SAFETY: Set to 4 to prevent 'Out of Memory'
        device=0,                 # Use your RTX 3050
        workers=2,                # Keep low for Windows stability
        project='BrainTumor_BiFPN', # Folder name for results
        name='run1',              # Sub-folder name
        optimizer='AdamW',        # Best optimizer for transformers/attention
        lr0=0.001,                # Initial learning rate
        cos_lr=True,              # Cosine scheduler for smooth convergence
        amp=True,                 # Automatic Mixed Precision (Faster, less VRAM)
        exist_ok=True             # Overwrite existing run folder if needed
    )
    
    print("Training Complete!")

if __name__ == '__main__':
    # This line is REQUIRED for Windows to prevent crashing
    torch.multiprocessing.freeze_support()
    train_custom_model()

Using Device: NVIDIA GeForce RTX 3050 Laptop GPU
Loading pre-trained weights...
Transferred 245/548 items from pretrained weights
Ultralytics 8.3.233  Python-3.11.14 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-bifpn.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=run1, 

In [ ]:
# 1. Load your best model
model = YOLO('BrainTumor_BiFPN/run1/weights/best.pt')

# 2. Get a random image from the test set
test_images = glob.glob('yolo_dataset/images/test/*.jpg')
if not test_images:
    print("No images found in test folder!")
    exit()

# Pick 3 random images to test
for i in range(3):
    img_path = random.choice(test_images)
    
    # 3. Predict
    # conf=0.4 means "Only show me if you are 40% sure"
    results = model.predict(img_path, conf=0.4, save=True) 

    # 4. Show Result
    for result in results:
        res_plotted = result.plot()
        cv2.imshow(f"Result {i+1}", res_plotted)

print("Press any key to close the image windows...")
cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 d:\Data Adji\Binus\Semester 5\DeepLearning\Project\yolo_dataset\images\test\brisc2025_test_00738_pi_ax_t1.jpg: 640x640 1 brain_tumor, 70.0ms
Speed: 8.4ms preprocess, 70.0ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 640)
Results saved to D:\Data Adji\Binus\Semester 5\DeepLearning\Project\runs\detect\predict2

image 1/1 d:\Data Adji\Binus\Semester 5\DeepLearning\Project\yolo_dataset\images\test\brisc2025_test_00224_gl_sa_t1.jpg: 640x640 1 brain_tumor, 65.3ms
Speed: 7.5ms preprocess, 65.3ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to D:\Data Adji\Binus\Semester 5\DeepLearning\Project\runs\detect\predict2

image 1/1 d:\Data Adji\Binus\Semester 5\DeepLearning\Project\yolo_dataset\images\test\brisc2025_test_00517_me_sa_t1.jpg: 640x640 1 brain_tumor, 64.5ms
Speed: 8.6ms preprocess, 64.5ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to D:\Data Adji\Binus\Semester 5\DeepLearning\Project\runs\de

In [ ]:
# Load your trained weights
model = YOLO('BrainTumor_BiFPN/run1/weights/best.pt')

# Export to ONNX (opset=12 is most compatible for web)
# 'dynamic=False' helps keeping it simpler for basic inference
model.export(format='onnx', opset=12, dynamic=False)

print("Exporting model 'best.onnx' in weights folder.")

Ultralytics 8.3.233  Python-3.11.14 torch-2.5.1 CPU (AMD Ryzen 7 4800H with Radeon Graphics)
YOLO11n-bifpn summary (fused): 112 layers, 2,591,477 parameters, 0 gradients, 6.4 GFLOPs

PyTorch: starting from 'BrainTumor_BiFPN\run1\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.2 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<=1.19.1', 'onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
    --------------------------------------- 0.3/16.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/16.5 MB 1.4 MB/s eta 0:00:12
   - -------------------------------------- 0.5/16.5 MB 1.4 MB/s eta 0:00:12
   - -------------------------------------- 0.8/16.5 MB 1.1 MB/s eta 0:00:14
   -- ------------------------------------- 1.0/16.5 MB 986.7 kB/s eta 0:00:16
   --- ------------------------------------ 1.3/16.5 MB 1.1 MB/s eta 0:00:15
   ---